In [1]:
from despliegue.loaders import *
from despliegue.nodos import *
from despliegue.contenedores import *
from despliegue.solvers import *
import pandas as pd
from os import path

Se definen las rutas de los datos, y también el `DataFrame` de la Red Móvil, ya que no posee la vacancia, y hay que asignarla a mano.

In [2]:
PATH_DATA = path.join("./data/")
PATH_CLIENTES = path.join(PATH_DATA, "Direcciones_Colina.xlsx")
PATH_FO = path.join(PATH_DATA, "CTO_Colina.xlsx")
PATH_RM = path.join(PATH_DATA, "SITIOS 4G TDD.xlsx")

In [3]:
df_clientes = pd.read_excel(PATH_CLIENTES)
df_clientes = df_clientes[df_clientes.gl_lat_OK <= -1]  # limpiamos aquellos que sean de lat-lon=0
df_clientes.describe()

,pcm_area_tel,gl_lat_OK,gl_lon_OK
count,2.750000e+02,275.000000,275.000000
mean,2.286022e+08,-33.200679,-70.672530
std,1.034212e+06,0.013769,0.011696
min,2.240048e+08,-33.301086,-70.723406
25%,2.284422e+08,-33.207438,-70.679538
50%,2.284444e+08,-33.202714,-70.673384
75%,2.284477e+08,-33.192004,-70.666531
max,2.329671e+08,-33.173324,-70.632150


In [4]:
df_fo = pd.read_excel(PATH_FO)
df_fo.describe()

,id_pto_ftth,latitud_ok,longitud_ok,eqpt_capacity,eqpt_vg_qty,oc_ds,porcentaje_ocup_caja
count,3.484000e+03,3484.000000,3484.000000,3484.000000,3484.00000,3484.00000,3484.000000
mean,2.476223e+06,-33.202056,-70.673801,7.756889,5.74225,2.00287,23.659374
std,4.067262e+05,0.013274,0.009063,0.839911,1.56150,1.40341,17.531295
min,1.758698e+06,-33.222792,-70.711280,1.000000,1.00000,0.00000,0.000000
25%,2.120851e+06,-33.214408,-70.681072,8.000000,5.00000,1.00000,12.500000
50%,2.478504e+06,-33.203207,-70.673720,8.000000,6.00000,2.00000,25.000000
75%,2.828275e+06,-33.190197,-70.667940,8.000000,7.00000,3.00000,37.500000
max,3.177018e+06,-33.176260,-70.649690,9.000000,8.00000,5.00000,57.140000


In [5]:
df_rm = pd.read_excel(PATH_RM)
df_rm["vacancia"] = 30
df_rm.describe()

,LATITUD,LONGITUD,vacancia
count,99.000000,99.000000,99.0
mean,-33.504739,-70.714568,30.0
std,0.116812,0.102712,0.0
min,-33.810990,-71.225640,30.0
25%,-33.576260,-70.746720,30.0
50%,-33.509860,-70.695800,30.0
75%,-33.409805,-70.664650,30.0
max,-33.205890,-70.537380,30.0


Creamos las instancias de conexión a la base de datos. Observar que para `rm_db` le entregamos el `DataFrame` directamente.

In [6]:
cliente_db = ClienteDB(df=df_clientes, col_names=["pcm_area_tel", "gl_lat_OK", "gl_lon_OK"])
fo_db = OfertaDB(df=df_fo, col_names=["id_pto_ftth", "latitud_ok", "longitud_ok", "eqpt_vg_qty"])
rm_db = OfertaDB(df=df_rm)

Creamos los conjuntos de nodos

In [7]:
nodos_oferta = NodosOferta(fo_db, rm_db)
nodos_demanda = NodosDemanda(cliente_db)

In [8]:
# modelo1 = Solver1(nodos_oferta, nodos_demanda, verbose=Tru
# modelo.construir_modelo()
# modelo.resolver()
# modelo.variables

In [9]:
# modelo2 = Solver2(nodos_oferta, nodos_demanda, verbose=True)
# modelo2.construir_modelo()
# modelo2.resolver()
# modelo2.variables

In [10]:
modelo3 = Solver3(nodos_oferta, nodos_demanda, verbose=True)
modelo3.construir_modelo()
modelo3.resolver()
modelo3.variables

Construyendo modelo...
Empezando a resolver...
Estado: Optimal


[x_(10,_150),
 x_(10,_175),
 x_(10,_225),
 x_(10,_83),
 x_(10,_85),
 x_(10,_98),
 x_(100,_111),
 x_(100,_264),
 x_(100,_274),
 x_(100,_95),
 x_(1000,_210),
 x_(1000,_57),
 x_(1001,_16),
 x_(1001,_239),
 x_(1001,_255),
 x_(1001,_30),
 x_(1001,_61),
 x_(1003,_232),
 x_(1004,_190),
 x_(1004,_257),
 x_(1007,_10),
 x_(1007,_35),
 x_(1007,_81),
 x_(1008,_187),
 x_(1008,_59),
 x_(101,_107),
 x_(101,_16),
 x_(101,_255),
 x_(101,_30),
 x_(1013,_115),
 x_(1013,_260),
 x_(1016,_117),
 x_(1016,_9),
 x_(1019,_148),
 x_(102,_187),
 x_(102,_199),
 x_(102,_47),
 x_(1022,_209),
 x_(1022,_250),
 x_(1022,_79),
 x_(1027,_121),
 x_(1030,_41),
 x_(1030,_50),
 x_(1030,_57),
 x_(1031,_196),
 x_(1032,_20),
 x_(1032,_266),
 x_(1035,_209),
 x_(1035,_250),
 x_(1036,_103),
 x_(1036,_130),
 x_(1036,_166),
 x_(1036,_195),
 x_(1036,_229),
 x_(1036,_65),
 x_(1040,_107),
 x_(1040,_255),
 x_(1040,_30),
 x_(1041,_79),
 x_(1043,_36),
 x_(1043,_45),
 x_(1046,_148),
 x_(1048,_89),
 x_(105,_130),
 x_(105,_27),
 x_(1054,_209)